# Welcome to the Lab 🥼🧪

## Rental Mix Analysis

Why does rental housing mix matter? The types of units trading varies by market and can provide insight into where rental prices are going. This analysis will look at the rental mix of a market. The [Parcl Labs Rental Price Feeds](https://www.parcllabs.com/articles/parcl-labs-rental-price-feed-white-paper) is the rental price per square foot of units trading on a market. Understanding the mix and the variation in mix over time can provide insight into where prices could go. 

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/traders/rental_mix_analysis.ipynb)

In [ ]:
# Environment setup
import os
import sys
import subprocess
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido numpy
    !git clone https://github.com/ParclLabs/parcllabs-examples.git
    sys.path.append('/content/parcllabs-examples/python/')
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')
    cur_dir = os.getcwd()
    chart_dir = os.path.join(cur_dir, '..')
    sys.path.append(chart_dir)

In [ ]:
import parcllabs
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from charting.utils import create_labs_logo_dict, format_metro_names

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# init client
client = ParclLabsClient(api_key=api_key)

In [ ]:
# Get top 100 CBSAs by population
markets = client.search_markets.retrieve(
    as_dataframe=True,
    # sort_by='PARCL_EXCHANGE_MARKET',
    sort_by='PRICEFEED_MARKET',
    sort_order='DESC',
    params={
        'limit': 100
    }
)

markets = markets.loc[markets['pricefeed_market']==1]

In [ ]:
def clean_names(nme):
    replace = {
        'Washington City': 'D.C.',
        'United States Of America': 'USA',
        'New York City': 'NYC',
        'Kings County': 'Brooklyn, NY',
    }
    if nme in replace.keys():
        return replace[nme]
    else:
        return nme
    
markets['name'] = markets['name'].apply(clean_names)
markets

In [ ]:
START_DATE = '2020-01-01'

rentals = client.rental_price_feed.retrieve_many(
        parcl_ids=markets['parcl_id'].tolist(),
        start_date=START_DATE,
        as_dataframe=True,
        params={'limit': 1000},  # expand the limit to 1000, these are daily series
        auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)

rentals.head()

In [ ]:
rentals = rentals.merge(markets[['name', 'parcl_id']], on='parcl_id', how='inner')

In [ ]:
yields = client.rental_market_metrics_gross_yield.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    property_type='ALL_PROPERTIES',
    as_dataframe=True,
    start_date=START_DATE,
    params={
        'limit': 300
    }
)

mkt_prices = client.market_metrics_housing_event_prices.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    as_dataframe=True,
    start_date=START_DATE,
    params={
        'limit': 300
    }
)

mkt_counts = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    as_dataframe=True,
    start_date=START_DATE,
    params={
        'limit': 300
    }
)

In [ ]:
rental_supply = client.rental_market_metrics_new_listings_for_rent_rolling_counts.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    as_dataframe=True,
    start_date='2023-03-01',
    params={
        'limit': 300
    }
)

rental_supply_sfh = client.rental_market_metrics_new_listings_for_rent_rolling_counts.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    property_type='SINGLE_FAMILY',
    as_dataframe=True,
    start_date='2023-03-01',
    params={
        'limit': 300
    }
)

rental_concentration = client.rental_market_metrics_rental_units_concentration.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    as_dataframe=True,
    start_date=START_DATE,
    params={
        'limit': 300
    }
)

In [ ]:
rental_supply.head()

In [ ]:
rental_supply_sfh = rental_supply_sfh.rename(columns={'rolling_30_day': 'rolling_30_day_sfh', 'rolling_60_day': 'rolling_60_day_sfh', 'rolling_90_day': 'rolling_90_day_sfh'})
rental_supply = rental_supply.merge(rental_supply_sfh[['parcl_id', 'date', 'rolling_30_day_sfh', 'rolling_60_day_sfh', 'rolling_90_day_sfh']], on=['parcl_id', 'date'], how='inner')


In [ ]:
# get investor metrics
investor_ownership = client.investor_metrics_housing_stock_ownership.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    as_dataframe=True,
    start_date=START_DATE,
    params={
        'limit': 300
    }
)

investor_prices = client.investor_metrics_housing_event_prices.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    as_dataframe=True,
    start_date=START_DATE,
    params={
        'limit': 300
    }
)

investor_counts = client.investor_metrics_housing_event_counts.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    as_dataframe=True,
    start_date=START_DATE,
    params={
        'limit': 300
    }
)

In [ ]:
housing_stock = client.market_metrics_housing_stock.retrieve_many(
    parcl_ids=rentals['parcl_id'].unique().tolist(),
    as_dataframe=True,
    start_date=START_DATE,
    params={
        'limit': 300
    }
)

In [ ]:
# rental_concentration['date'] = pd.to_datetime(rental_concentration['date'])
rental_concentration = rental_concentration.rename(columns={'date': 'month_start'})
# housing_stock['date'] = pd.to_datetime(housing_stock['date'])
housing_stock = housing_stock.rename(columns={'date': 'month_start'})

In [ ]:
rental_supply['date'] = pd.to_datetime(rental_supply['date'])
rental_supply['month_start'] = rental_supply['date'].dt.to_period('M').dt.to_timestamp()

rental_supply.head()

In [ ]:
pf = rentals.copy(deep=True)
pf['date'] = pd.to_datetime(pf['date'])
pf['month_start'] = pf['date'].dt.to_period('M').dt.to_timestamp()
pf_monthly = pf.groupby(['parcl_id', 'month_start'])['rental_price_feed'].median().reset_index()
investor_ownership_cpy = investor_ownership.copy(deep=True)
investor_ownership_cpy['date'] = pd.to_datetime(investor_ownership_cpy['date'])
investor_ownership_cpy = investor_ownership_cpy.rename(columns={'date': 'month_start'})
pf_monthly = pf_monthly.merge(investor_ownership_cpy, on=['parcl_id', 'month_start'], how='inner')
pf_monthly['pf_shift'] = pf_monthly['rental_price_feed'].shift(-4)
pf_monthly[['investor_pct_ownership', 'pf_shift']].corr()

In [ ]:

cpy = investor_ownership.copy(deep=True)
cpy['date'] = pd.to_datetime(cpy['date'])
cpy = cpy.rename(columns={'date': 'month_start'})
chart = rental_supply.merge(cpy, on=['parcl_id', 'month_start'], how='inner')
chart = chart.merge(rental_concentration, on=['parcl_id', 'month_start'], how='inner')
chart = chart.merge(housing_stock, on=['parcl_id', 'month_start'], how='inner')
chart.head()

In [ ]:
chart['rolling_60_day_all_other'] = chart['rolling_60_day'] - chart['rolling_60_day_sfh']
chart['all_other_properties'] = chart['all_properties'] - chart['single_family']

In [ ]:
markets.sample(n=5)

In [ ]:
pid = 5387853
markets.loc[markets['parcl_id'] == pid]

In [ ]:
chart['vacancy_rate'] = chart['rolling_60_day'] / chart['rental_units']
chart['rental_stock_percentage'] = chart['rolling_60_day'] / chart['all_properties']
chart['rental_stock_percentage_sfh'] = chart['rolling_60_day_sfh'] / chart['single_family']
chart['rental_stock_percentage_all_other'] = chart['rolling_60_day_all_other'] / chart['all_other_properties']
chart.loc[chart['parcl_id']==pid].plot(x='date', y='investor_pct_ownership', title='New Listings for Rent', figsize=(10, 5))

In [ ]:
rentals.loc[(rentals['parcl_id']==pid) & (rentals['date']>='2020-03-01')].plot(x='date', y='rental_price_feed', title='Rental Price Feed', figsize=(10, 5))

In [ ]:
investor_ownership.head()

In [ ]:
chart2 = chart.merge(rentals, on=['parcl_id', 'date'], how='inner')
chart2['pf_shift'] = chart2.groupby('parcl_id')['rental_price_feed'].shift(-8)
chart2 = chart2.dropna(subset=['pf_shift'])
chart2.head()

In [ ]:
test = chart2.loc[chart2['parcl_id']==pid]
test['rental_stock_percentage_change'] = test['rental_stock_percentage'].pct_change()
test['rental_stock_percentage_sfh_change'] = test['rental_stock_percentage_sfh'].pct_change()
test['rental_stock_percentage_all_other_change'] = test['rental_stock_percentage_all_other'].pct_change()
test['rolling_7_day_change'] = test['rolling_7_day'].pct_change()
test['rolling_30_day_change'] = test['rolling_30_day'].pct_change()
test['rolling_60_day_change'] = test['rolling_60_day'].pct_change()
test['rolling_90_day_change'] = test['rolling_90_day'].pct_change()
test['vacancy_change'] = test['vacancy_rate'].pct_change()
test = test.dropna()

In [ ]:
test[['investor_pct_ownership', 'pf_shift']].corr()

In [ ]:
chart2[['rolling_60_day', 'pf_shift']].corr()

In [ ]:
chart2.loc[chart2['pct_rental_concentration'] > 40][['investor_pct_ownership', 'pf_shift']].corr()

In [ ]:
chart2

In [ ]:
chart2.loc[chart2['pct_rental_concentration']>40]['name'].unique()

In [ ]:
chart2[['investor_pct_ownership', 'pf_shift']].corr()

In [ ]:
nme = chart2.groupby('name')['pct_rental_concentration'].mean().reset_index()
nme.loc[nme['name'] == 'Chicago City']

In [ ]:
markets.loc[markets['name'] == 'Chicago City']

In [ ]:
nme.loc[nme['pct_rental_concentration'] > 30].sort_values('pct_rental_concentration', ascending=False)

In [ ]:
chart2.loc[chart2['name']=='Miami City']

In [ ]:
# feature engineering, data prep
investor_counts = investor_counts.rename(columns={
    'acquisitions': 'investor_acquisitions',
    'dispositions': 'investor_dispositions',
    'new_listings_for_sale': 'investor_new_listings_for_sale'
})

investor_ownership = investor_ownership.rename(columns={
    'count': 'investor_count',
    'pct_ownership': 'investor_pct_ownership'
})

investor_prices = investor_prices.rename(columns={
    'price_per_square_foot_median_acquisitions': 'investor_price_per_square_foot_median_acquisitions',
    'price_per_square_foot_median_new_listings_for_sale': 'investor_price_per_square_foot_median_new_listings_for_sale'
})

investor_counts['investor_net'] = investor_counts['investor_acquisitions'] - investor_counts['investor_dispositions']
mkt_prices = mkt_prices.rename(
    columns=
    {
        'price_median_sales': 'mkt_price_median_sales', 
        'price_median_new_listings_for_sale': 'mkt_price_median_new_listings_for_sale',
        'price_per_square_foot_median_sales': 'mkt_price_per_square_foot_median_sales',
        'price_per_square_foot_median_new_listings_for_sale': 'mkt_price_per_square_foot_median_new_listings_for_sale'
    })

In [ ]:
# mkt_counts = mkt_counts.drop('property_type', axis=1)

In [ ]:
investor_prices.head()

In [ ]:

tmp = yields.merge(investor_counts[['date', 'parcl_id', 'investor_acquisitions', 'investor_dispositions', 'investor_new_listings_for_sale']], on=['parcl_id', 'date'])
tmp = tmp.merge(
    investor_prices[[
        'parcl_id',
        'date',
        'investor_price_per_square_foot_median_acquisitions', 
        'investor_price_per_square_foot_median_new_listings_for_sale'
    ]], 
    on=['parcl_id', 'date']
)
tmp = tmp.merge(investor_ownership, on=['parcl_id', 'date'])
tmp = tmp.merge(mkt_counts, on=['parcl_id', 'date'])
tmp = tmp.merge(markets[['location_type', 'parcl_id']], on='parcl_id')
tmp = tmp.merge(mkt_prices[['mkt_price_per_square_foot_median_sales', 'date', 'parcl_id']], on=['parcl_id', 'date'])
# tmp = tmp.merge(supply, on=['parcl_id', 'date'])
# tmp[['pct_gross_yield', 'net']].corr()

# Ensure 'date' column is in datetime format
rentals['date'] = pd.to_datetime(rentals['date'])

# Truncate date to the first of the month
rentals['month_start'] = rentals['date'].dt.to_period('M').dt.to_timestamp()
rentals.head()

agg = rentals.groupby(['parcl_id', 'month_start'])['rental_price_feed'].mean().reset_index()
agg = agg.rename(columns={'month_start': 'date'})
tmp['date'] = pd.to_datetime(tmp['date'])
tmp = tmp.merge(agg, on=['parcl_id', 'date'], how='inner')
tmp = tmp.sort_values('date')
tmp['pf_shift'] = tmp.groupby('parcl_id')['rental_price_feed'].shift(-3)
tmp = tmp.dropna()
tmp.head()

In [ ]:
tmp.loc[tmp['parcl_id']==5372594][['date', 'rental_price_feed', 'pf_shift']]

In [ ]:


# Exploratory Data Analysis (EDA)
plt.figure(figsize=(12, 6))
sns.heatmap(
    tmp[[
        'pct_gross_yield',
'investor_acquisitions',
 'investor_dispositions',
 'investor_new_listings_for_sale',
 'investor_price_per_square_foot_median_acquisitions',
 'investor_price_per_square_foot_median_new_listings_for_sale',
 'investor_count',
 'investor_pct_ownership',
 'sales',
 'new_listings_for_sale',
 'new_rental_listings',
 'mkt_price_per_square_foot_median_sales',
 'rental_price_feed',
 'pf_shift'
]].corr(), 
    annot=True, 
    cmap='coolwarm')
plt.title('Feature Correlation Matrix')
plt.show()

In [ ]:
# Assume tmp is your DataFrame
tmp['date'] = pd.to_datetime(tmp['date'])
tmp['year'] = tmp['date'].dt.year
tmp['month'] = tmp['date'].dt.month
tmp['parcl_id'] = tmp['parcl_id'].astype('category')

tmp['season'] = tmp['month'].apply(lambda x: 'Winter' if x in [12, 1, 2] else 'Spring' if x in [3, 4, 5] else 'Summer' if x in [6, 7, 8] else 'Fall')

In [ ]:
tmp['investor_ppsqf_purchase_premium'] = (tmp['investor_price_per_square_foot_median_acquisitions'] - tmp['mkt_price_per_square_foot_median_sales'])/tmp['mkt_price_per_square_foot_median_sales']

In [ ]:
# Define numerical features
numerical_features = [
'pct_gross_yield',
# 'investor_acquisitions',
 # 'investor_dispositions',
 # 'investor_new_listings_for_sale',
 'investor_price_per_square_foot_median_acquisitions',
 # 'investor_price_per_square_foot_median_new_listings_for_sale',
 # 'investor_count',
 'investor_pct_ownership',
 'sales',
 # 'new_listings_for_sale',
# 'new_rental_listings',
 'mkt_price_per_square_foot_median_sales',
 # 'investor_ppsqf_purchase_premium'
]
categorical_features = ['season']

# Checking multicollinearity using VIF
X_vif = tmp[numerical_features].dropna()

# Apply RobustScaler to handle outliers and extreme values
scaler = RobustScaler()
X_vif_scaled = scaler.fit_transform(X_vif)

vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif_scaled, i) for i in range(len(X_vif.columns))]
print(vif_data)

# Creating train-test split
X = tmp[numerical_features + categorical_features]
y = tmp['pf_shift']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])

# Ridge Regression Model
ridge_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])
ridge_model.fit(X_train, y_train)

# Predict
y_ridge_pred = ridge_model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_ridge_pred)
mse = mean_squared_error(y_test, y_ridge_pred)
ridge_r2 = r2_score(y_test, y_ridge_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2: {ridge_r2}')

# Cross-validation
cv_scores = cross_val_score(ridge_model, X, y, cv=TimeSeriesSplit(n_splits=5), scoring='r2')
print(f'Cross-validated R2 scores: {cv_scores}')
print(f'Mean cross-validated R2 score: {np.mean(cv_scores)}')

# Feature importance
feature_names = numerical_features + list(ridge_model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features))
ridge_coefs = ridge_model.named_steps['regressor'].coef_

feature_importance = pd.Series(ridge_coefs, index=feature_names).sort_values(ascending=False)
plt.figure(figsize=(12, 8))
feature_importance.plot(kind='bar')
plt.title('Feature Importance - Ridge Regression')
plt.show()

# Residual analysis
residuals = y_test - y_ridge_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_ridge_pred, residuals)
plt.axhline(0, color='r', linestyle='--')
plt.title('Residuals vs Predicted')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.show()


In [ ]:
# explore the relationship between yields and investor pct ownership

a = tmp[['date', 'pct_gross_yield', 'investor_acquisitions']].copy()
a['date'] = pd.to_datetime(a['date'])
a = a.set_index('date')
a.head()

In [ ]:
a.corr()

In [ ]:
# shift investor_pct_ownership by 3 months
a['investor_acquisitions_shifted'] = a['investor_acquisitions'].shift(3)
a = a.dropna()
a.corr()

In [ ]:
tmp.head()

In [ ]:
tmp.columns.tolist()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests

# Function to create lagged features
def create_lagged_features(df, features, max_lag):
    lagged_features = pd.DataFrame(index=df.index)
    for feature in features:
        for lag in range(1, max_lag + 1):
            lagged_features[f'{feature}_lag{lag}'] = df.groupby('parcl_id')[feature].shift(lag)
    return lagged_features

# Define your features and max lag
numerical_features = [
    'pct_gross_yield',
    'investor_acquisitions',
    'investor_dispositions',
    'investor_new_listings_for_sale',
    'investor_price_per_square_foot_median_acquisitions',
    'investor_price_per_square_foot_median_new_listings_for_sale',
    'investor_count',
    'investor_pct_ownership',
    'sales',
    'new_listings_for_sale',
    'new_rental_listings',
    'mkt_price_per_square_foot_median_sales',
    'rental_price_feed',
    'investor_ppsqf_purchase_premium'
]
max_lag = 12

# Create lagged features
lagged_df = create_lagged_features(tmp, numerical_features, max_lag)

# Merge lagged features with the original dataframe
lagged_df = tmp.join(lagged_df).dropna()

# Function to compute cross-correlation
def compute_cross_correlation(df, x, y, max_lag):
    correlations = []
    for lag in range(1, max_lag + 1):
        corr = df[x].corr(df[f'{y}_lag{lag}'])
        correlations.append((x, y, lag, corr))
    return correlations

# Function to perform Granger causality test
def granger_test(df, x, y, max_lag):
    test_result = grangercausalitytests(df[[x, y]].dropna(), max_lag, verbose=False)
    p_values = [round(test_result[i + 1][0]['ssr_ftest'][1], 4) for i in range(max_lag)]
    return p_values

# Initialize DataFrames to store results
cross_corr_results = pd.DataFrame(columns=['Variable_X', 'Variable_Y', 'Lag', 'Correlation'])
granger_results = pd.DataFrame(columns=['Variable_X (Lagged)', 'Variable_Y', 'Lag', 'P_Value'])

# Loop through all combinations of variables to perform cross-correlation and Granger causality tests
for x in numerical_features:
    for y in numerical_features:
        if x != y:
            # Compute cross-correlation
            cross_corr = compute_cross_correlation(lagged_df, x, y, max_lag)
            cross_corr_df = pd.DataFrame(cross_corr, columns=['Variable_X', 'Variable_Y', 'Lag', 'Correlation'])
            cross_corr_results = pd.concat([cross_corr_results, cross_corr_df], ignore_index=True)
            
            # Perform Granger causality test
            granger_p_values = granger_test(lagged_df, x, y, max_lag)
            granger_df = pd.DataFrame({
                'Variable_X (Lagged)': [f'{x}_lag{i+1}' for i in range(max_lag)], 
                'Variable_Y': [y]*max_lag, 
                'Lag': list(range(1, max_lag + 1)), 
                'P_Value': granger_p_values
            })
            granger_results = pd.concat([granger_results, granger_df], ignore_index=True)

# Review and analyze the results
# Cross-correlation: Filter strong correlations
strong_correlations = cross_corr_results[cross_corr_results['Correlation'].abs() > 0.5]
print("Strong Cross-Correlations:")
print(strong_correlations)

# Granger causality: Filter significant p-values (e.g., p < 0.05)
significant_granger = granger_results[granger_results['P_Value'] < 0.05]
print("Significant Granger Causality Results:")
print(significant_granger)

# Save results to CSV files for further analysis if needed
cross_corr_results.to_csv('cross_correlation_results.csv', index=False)
granger_results.to_csv('granger_causality_results.csv', index=False)


In [ ]:
significant_granger['Variable_Y'].unique()

In [ ]:
lagged_df.shape

In [ ]:
tmp.shape

In [ ]:
strong_correlations.head(100)

In [ ]:
significant_granger.to_csv('granger.csv')